# Converting Pdf to Image

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import time
import cv2
import fitz
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

start = time.time()

in_path = '/home/vimalkumar/Documents/softco/test/1st page/pdf'
out_path = '/home/vimalkumar/Documents/softco/test/1st page/images'

for file in os.listdir(in_path):
    #print(file)
    file_name = os.path.splitext(file)[0]
    if file.endswith('.tif') or file.endswith('.TIF') or file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.gif') or file.endswith('.jpg') or file.endswith('.JPG')or file.endswith('.webp'):
        #print(file)
        #print(file_name)
        img = Image.open(os.path.join(in_path,file))
        img.seek(0)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(os.path.join(out_path,file_name +'.jpeg'))
        
    if file.endswith('.pdf') or file.endswith('.PDF'):
        #print(file)
        doc = fitz.open(os.path.join(in_path,file))
        #print(len(doc))
        mat = fitz.Matrix(300 / 72, 300 / 72)
        page = doc.load_page(0)  # number of page
        pix = page.get_pixmap(matrix=mat)
        #print(type(pix))
        #plt.imshow(pix)
        
        pix.pil_save(os.path.join(out_path,file_name+'.jpeg'),format="JPEG", dpi=(300,300))


        
end = time.time()
print(f"Runtime of the program is {end - start}")
        

Runtime of the program is 11.423489809036255


# 1st - Single page


# Updated

In [5]:
import cv2
import os
import time
import fitz
import keras
import pytesseract
import pandas as pd
import numpy as np
from pytesseract import Output
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image

start = time.time()


model = keras.models.load_model('/home/vimalkumar/Documents/Python_new_1/SOFTCO/CNN Model/invoice_inception_model.h5')

dic = {0 : 'Invoice',1 : 'Unclassified'}
#print(dic)

filepath = '/home/vimalkumar/Documents/softco/test/1st page/images'
outpath = '/home/vimalkumar/Documents/softco/test/1st page/text'


file_name = []
result = []
thresh_list=[]

count = 1

for im in os.listdir(filepath):
    print('Document No:',count)
    count = count + 1
    img = image.load_img(os.path.join(filepath,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    img_arr = image.img_to_array(img)
    img_arr = img_arr/255
    img_arr = np.expand_dims(img_arr,axis=0)
    images = np.vstack([img_arr])
    val = model.predict(images)
    print(val)
    #print(val[0][1])

    a=np.argmax(val,axis=1)
    #print(dic[a[0]])
    
    if val[0][1] >= 0.95:
        print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
        #file_name.append(os.path.splitext(img)[0])
        file_name.append(os.path.splitext(im)[0]+'.pdf')
        result.append('UNCLASSIFIED')
        thresh_list.append('0')
        
    else:
        f_name = os.path.splitext(im)[0]
        imm = cv2.imread(os.path.join(filepath,im),0)
        image_data = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                               config=r'--oem 3 --psm 6')
        #print(image_data)


        text=[]
        line_num=[]
        output=[]
        for i in image_data["text"]:
            text.append(i)

        for j in image_data["line_num"]:
            line_num.append(j)

        new_list=zip(line_num,text)
        for i in new_list:
        #     print(i)
            if i[0]>= 0:
                output.append(i[1])

        # print(output)
        new=" ".join(output)
        
        image_data_1 = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                       config=r'--oem 3 --psm 11')
        #print(image_data)


        text_1=[]
        line_num_1=[]
        output_1=[]
        for i in image_data_1["text"]:
            text_1.append(i)

        for j in image_data_1["line_num"]:
            line_num_1.append(j)

        new_list_1=zip(line_num_1,text_1)
        for i in new_list_1:
        #     print(i)
            if i[0]>= 0:
                output_1.append(i[1])

        # print(output)
        new_1=" ".join(output_1)
        #print(new_1)

        final_new = new + new_1
        #print(final_new)

        text_file = open(os.path.join(outpath,f_name + '.txt'), 'w')
        text_file.write(final_new)
        text_file.close()

        #print(new)
        text = final_new.lower()
        #print(text)
        
        data_df = pd.DataFrame(image_data)
        
        cred_list = []
        for m in range(25):
            df_text = data_df[data_df['line_num']==m]

            for n in df_text['text']:
                cred_list.append(n)
        cred_list = ' '.join(cred_list).split()
        #print(cred_list)
        cred_out = ''
        for r in cred_list:
            cred_out= cred_out + ' ' + r
        
        cred_out = cred_out.lower()
        #print(cred_out)
        
        data_df_1 = pd.DataFrame(image_data_1)

        cred_list_1 = []
        for m in range(25):
            df_text_1 = data_df_1[data_df_1['line_num']==m]

            for n in df_text_1['text']:
                cred_list_1.append(n)
        cred_list_1 = ' '.join(cred_list_1).split()
        #print(cred_list_1)
        cred_out_1 = ''
        for r in cred_list_1:
            cred_out_1= cred_out_1 + ' ' + r

        cred_out_1 = cred_out_1.lower()
        #print(cred_out_1)

        final_cred = cred_out + cred_out_1
        
        keywords = ['Invoice','Invoice_Date','Invoice_Num','Description','VAT','Unit',
            'Deliver_Num','Order_Num','Amount']
           
        Invoice = ['INVOICE']

        Invoice_Date = ['Invoice Date','Inv Date','Invoice / Tax Date','DATE/TAX POINT',
                       'Date of invoice']

        Invoice_Num = ['Invoice Number','Invoice Num','Invoice No','Inv No','Invoice #',
                       'Invoice#','Invoice ID','Invoice n°','Invoice N','InvoiceN',
                       'Invoice ref','InvoiceRef','Invoice Nr','invoice-no','Sales - Invoice',
                      'Sales-Invoice','InvoiceID','Invoice Customer','invoice-n',
                      'invoice :','invoice:','number/date']

        Description = ['Description','Product','Details','Materials','Item','Summary',
                       'ACTIVITY','Narrative','Material']

        VAT = ['VAT','TAX','V A T','V.A.T']

        Unit = ['Unit','Quantity','QTY']

        Deliver_Num = ['Deliver Number','Delivery Note','Delivery No','DelNum',
                       'Despatch No',
                      'Delivery Note Number','Del Note','Delivery Order','Delivery Note No']

        Order_Num = ['Order Number','Order No','Order Num','your order','your reference',
                    'Order Ref','Customer Ref',"Buyer's reference",'Customer Reference',
                    'order reference','']

        Amount = ['Balance Due','Total Due','Invoice Balance Due','Total Amount Due',
                  'Current invoice balance','Amount Due (USD)','Total in USD',
                  'Total Amount (GBP)','invoice total','TOTAL USD','TOTAL (USD)',
                    'amount due', 'Total EUR','Total (EUR)','Total (GBP)','Invoice subtotal',
                 'USD Total']

        credit_note = ['Credit Note','Credit for Returns','CREDIT-NOTE','ORIGINAL CREDIT MEMO','CREDIT_NOTE_ALLOWANCE','Credit note','CREDIT NOTE(COPY)','CREDIT MEMO','Credit Memo','CreditNote',
                        'Return Credit','CREDIT NOTE','RETURN CREDIT MEMO','Credit Reference:',
                       'Cr.Note No','Credit (Copy)','SALES CREDIT NOTE','credit date',
                       'Credit Note No.','Credit Note Number:','Credit Note No.',
                       'RETURN CREDIT','Credit memo','Creditmemo','Cr.Note Date','sales credit']
        inv = []
        inv_date = []
        inv_num = []
        des = []
        vat = []
        unit = []
        del_num = []
        ord_num = []
        amt = []
        final_list = []

        for cr in credit_note:
            if cr.lower() in final_cred:
                print(os.path.splitext(im)[0]+'.pdf' , 'Credit Note')
                file_name.append(os.path.splitext(im)[0]+'.pdf')
                result.append('Credit Note')
                thresh_list.append('0')
                break


        else:

            for a in Invoice:
                if a.lower() in text:
                    #print(a)
                    inv.append(a)
                    final_list.append('Invoice')
                    break
                else:
                    inv.append('-')
            if len(inv)==inv.count('-'):
                final_list.append('nil')
                #print('-------')
            for b in Invoice_Date:
                if b.lower() in text:
                    #print(b)
                    inv_date.append(b)
                    final_list.append('Invoice_Date')
                    break
                else:
                    inv_date.append('-')
            if len(inv_date)==inv_date.count('-'):
                final_list.append('nil')
                #print('-------')
            for c in Invoice_Num:
                if c.lower() in text:
                    #print(c)
                    inv_num.append(c)
                    final_list.append('Invoice_Num')
                    break
                else:
                    inv_num.append('-')
            if len(inv_num)==inv_num.count('-'):
                #print('-------')
                final_list.append('nil')
            for d in Description:
                if d.lower() in text:
                    #print(d)
                    des.append(d)
                    final_list.append('Description')
                    break
                else:
                    des.append('-')
            if len(des)==des.count('-'):
                final_list.append('nil')
                #print('-------')
            for e in VAT:
                if e.lower() in text:
                    #print(e)
                    vat.append(e)
                    final_list.append('VAT')
                    break
                else:
                    vat.append('-')
            if len(vat)==vat.count('-'):
                final_list.append('nil')
                #print('-------')
            for f in Unit:
                if f.lower() in text:
                    #print(f)
                    unit.append(f)
                    final_list.append('Unit')
                    break
                else:
                    unit.append('-')
            if len(unit)==unit.count('-'):
                final_list.append('nil')
                #print('-------')

            for g in Deliver_Num:
                if g.lower() in text:
                    #print(g)
                    del_num.append(g)
                    final_list.append('Deliver_Num')
                    break
                else:
                    del_num.append('-')
            if len(del_num)==del_num.count('-'):
                final_list.append('nil')
                #print('-------')
            for o in Order_Num:
                if o.lower() in text:
                    #print(o)
                    ord_num.append(o)
                    final_list.append('Order_Num')
                    break
                else:
                    ord_num.append('-')
            if len(ord_num)==ord_num.count('-'):
                final_list.append('nil')
                #print('-------')
            for p in amt:
                if p.lower() in text:
                    #print(o)
                    amt.append(p)
                    final_list.append('Amount')
                    break
                else:
                    amt.append('-')
            if len(amt)==amt.count('-'):
                final_list.append('nil')
                #print('-------')

            thresh = [15,10,25,15,10,5,5,5,10]

            threshold = 0
            for i,j,k in zip(keywords,final_list,thresh):
                if i == j:
                    threshold = threshold + k

                #else:
                    #print('--no keyword in image--')

            #thresh_list.append(threshold)
            print(threshold)
            #print(final_list)
            
            if val[0][0] >= 0.9:
                threshold = threshold + 20
                thresh_list.append(threshold)
                if threshold >= 65:
                    print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('Invoice')
                else:
                    print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('UNCLASSIFIED')
                    
            elif val[0][0] >=0.7 and val[0][0] < 0.9:
                threshold = threshold + 10
                thresh_list.append(threshold)
                if threshold >= 60:
                    print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('Invoice')
                else:
                    print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('UNCLASSIFIED')
                    
            else:
                threshold = threshold + 0
                thresh_list.append(threshold)
                if threshold >= 60:
                    print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('Invoice')
                else:
                    print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('UNCLASSIFIED')

df = pd.DataFrame({'File_name':file_name,'Classified_Result':result,
                   'Threshold':thresh_list})
pd.set_option('display.max_rows',500)
display(df)

df.to_excel('/home/vimalkumar/Documents/softco/test/1st page/1stpage_classification_result')
                
end = time.time()

print(end-start)
                
                    

    
    





2022-03-03 09:37:45.403451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-03 09:37:45.403482: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-03 09:37:45.403499: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-03-03 09:37:45.403802: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Document No: 1


2022-03-03 09:37:52.771130: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


[[1.0000000e+00 1.1554528e-09]]
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820439.pdf Credit Note
Document No: 2
[[9.999685e-01 6.672336e-06]]
50
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820511.pdf INVOICE
Document No: 3
[[9.9961799e-01 1.6582446e-05]]
90
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820497.pdf INVOICE
Document No: 4
[[9.9999177e-01 2.1370724e-07]]
85
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820491.pdf INVOICE
Document No: 5
[[9.9987531e-01 4.0103296e-05]]
75
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820472.pdf INVOICE
Document No: 6
[[9.9985617e-01 1.5288388e-05]]
80
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820435.pdf INVOICE
Document No: 7
[[9.9986809e-01 4.7962447e-05]]
25
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820481.pdf UNCLASSIFIED
Document No: 8
[[9.9999321e-01 3.1791734e-07]]
50
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820477.pdf INVOICE
Document No: 9
[[9.9997699e-01 1.9351294e-06]]
60
ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820508.pdf INVOICE
Document No:

KeyboardInterrupt: 

# All pages 

# Updated 2.0

In [3]:
import cv2
import os
import time
import fitz
import keras
import pytesseract
import pandas as pd
import numpy as np
from pytesseract import Output
from PIL import Image
from termcolor import colored

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image

start = time.time()

in_path = '/home/vimalkumar/Documents/softco/test/pdf/testing-6/pdf'
out_path = '/home/vimalkumar/Documents/softco/test/pdf/testing-6/images'

file_name = []
result = []
thresh_list=[]

count_val = 1
for file in os.listdir(in_path):
    print(colored('Document : ','red', attrs=['bold']),count_val)
    #print('Document : ', count_val)
    count_val += 1
    #print(file)
#     os.makedirs(os.path.join(out_path,os.path.splitext(pdf)[0]))
    #os.makedirs(os.path.join(out_path,os.path.splitext(file)[0]))

#     for file in os.listdir(os.path.join(out_path,os.path.splitext(pdf)[0])):
#         #print(file)
    pdf_file_name = os.path.splitext(file)[0]
    if file.endswith('.tif') or file.endswith('.TIF') or file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.gif') or file.endswith('.jpg') or file.endswith('.JPG')or file.endswith('.webp'):
        #print(file)
        #print(file_name)
        img = Image.open(os.path.join(in_path,file))
        img.seek(0)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(os.path.join(out_path,os.path.splitext(file)[0],pdf_file_name +'.jpeg'))

    if file.endswith('.pdf') or file.endswith('.PDF'):
        print(file)
        
        os.makedirs(os.path.join(out_path,os.path.splitext(file)[0]))

        doc = fitz.open(os.path.join(in_path,file))
        print(colored('No of Pages : ','green', attrs=['bold']),len(doc))
        #print('No of Pages : ',len(doc))
        mat = fitz.Matrix(150 / 72, 150 / 72)
        for k in range(len(doc)):
            page = doc.load_page(k)  # number of page
            pix = page.get_pixmap(matrix=mat)
            #print(type(pix))
            #plt.imshow(pix)

            pix.pil_save(os.path.join(out_path,os.path.splitext(file)[0],pdf_file_name+'_'+str(k+1)+'.jpeg'))

        lst = sorted(os.listdir(os.path.join(out_path,os.path.splitext(file)[0])))
        lst.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

        #for image in lst:
            #print(image)

        model = keras.models.load_model('/home/vimalkumar/Documents/Python_new_1/SOFTCO/CNN Model/invoice_inception_model.h5')

        dic = {0 : 'Invoice',1 : 'Unclassified'}
        #print(dic)

        filepath = os.path.join(out_path,os.path.splitext(file)[0])
        outpath = '/home/vimalkumar/Documents/softco/test/pdf/testing-6/text'


    #     file_name = []
    #     result = []
    #     thresh_list=[]

        count = 1

        for im in lst:
            print(colored('page No:','blue', attrs=['bold']),count)
            #print('page No:',count)
            count = count + 1
            img = image.load_img(os.path.join(filepath,im),
                             target_size=(224,224))
            #plt.imshow(img)
            #plt.show()

            img_arr = image.img_to_array(img)
            img_arr = img_arr/255
            img_arr = np.expand_dims(img_arr,axis=0)
            images = np.vstack([img_arr])
            val = model.predict(images)
            print(val)
            #print(val[0][1])

            a=np.argmax(val,axis=1)
            #print(dic[a[0]])

            if val[0][1] >= 0.95:
                print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                #file_name.append(os.path.splitext(img)[0])
                file_name.append(os.path.splitext(im)[0]+'.pdf')
                result.append('UNCLASSIFIED')
                thresh_list.append('0')

            else:
                f_name = os.path.splitext(im)[0]
                imm = cv2.imread(os.path.join(filepath,im),0)
                image_data = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                                       config=r'--oem 3 --psm 6')
                #print(image_data)


                text=[]
                line_num=[]
                output=[]
                for i in image_data["text"]:
                    text.append(i)

                for j in image_data["line_num"]:
                    line_num.append(j)

                new_list=zip(line_num,text)
                for i in new_list:
                #     print(i)
                    if i[0]>= 0:
                        output.append(i[1])

                # print(output)
                new=" ".join(output)

                image_data_1 = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                               config=r'--oem 3 --psm 11')
                #print(image_data)


                text_1=[]
                line_num_1=[]
                output_1=[]
                for i in image_data_1["text"]:
                    text_1.append(i)

                for j in image_data_1["line_num"]:
                    line_num_1.append(j)

                new_list_1=zip(line_num_1,text_1)
                for i in new_list_1:
                #     print(i)
                    if i[0]>= 0:
                        output_1.append(i[1])

                # print(output)
                new_1=" ".join(output_1)
                #print(new_1)

                final_new = new + new_1
                #print(final_new)

                text_file = open(os.path.join(outpath,f_name + '.txt'), 'w')
                text_file.write(final_new)
                text_file.close()

                #print(new)
                text = final_new.lower()
                #print(text)

                data_df = pd.DataFrame(image_data)

                cred_list = []
                for m in range(15):
                    df_text = data_df[data_df['line_num']==m]

                    for n in df_text['text']:
                        cred_list.append(n)
                cred_list = ' '.join(cred_list).split()
                #print(cred_list)
                cred_out = ''
                for r in cred_list:
                    cred_out= cred_out + ' ' + r

                cred_out = cred_out.lower()
                #print(cred_out)

#                 data_df_1 = pd.DataFrame(image_data_1)

#                 cred_list_1 = []
#                 for m in range(3):
#                     df_text_1 = data_df_1[data_df_1['line_num']==m]

#                     for n in df_text_1['text']:
#                         cred_list_1.append(n)
#                 cred_list_1 = ' '.join(cred_list_1).split()
#                 #print(cred_list_1)
#                 cred_out_1 = ''
#                 for r in cred_list_1:
#                     cred_out_1= cred_out_1 + ' ' + r

#                 cred_out_1 = cred_out_1.lower()
#                 #print(cred_out_1)

                final_cred = cred_out# + cred_out_1
                #print(final_cred)

                keywords = ['Invoice','Invoice_Date','Invoice_Num','Description','VAT','Unit',
                    'Deliver_Num','Order_Num','Amount']

                Invoice = ['INVOICE']

                Invoice_Date = ['Invoice Date','Inv Date','Invoice / Tax Date','DATE/TAX POINT',
                               'Date of invoice',]

                Invoice_Num = ['Invoice Number','Invoice Num','Invoice No','Inv No','Invoice #',
                'Invoice#','Invoice ID','Invoice n°','Invoice N','InvoiceN',
                'Invoice ref','InvoiceRef','Invoice Nr','invoice-no','Sales - Invoice',
                'Sales-Invoice','InvoiceID','Invoice Customer','invoice-n',
                'invoice :','invoice:','number/date','Invoice Number:','invioce no',
                'invioce number','invioce num','invioce #','Invoice/Cr No',
                'Invoice/CrNo','invoice SI','invoiceSI','invoice  SI',
                'INVOICE Doc. No','INVOICE Doc.No','INVOICE Doc No','INVOICE DocNo',
                'invoice(copy)','invoice (copy)']

                Description = ['Description','Product','Details','Materials','Summary',
                               'ACTIVITY','Narrative','Material']

                VAT = ['VAT','TAX','V A T','V.A.T']

                Unit = ['Unit','Quantity','QTY']

                Deliver_Num = ['Deliver Number','Delivery Note','Delivery No','DelNum',
                               'Despatch No',
                              'Delivery Note Number','Del Note','Delivery Order','Delivery Note No']

                Order_Num = ['Order Number','Order No','Order Num','your order','your reference',
                            'Order Ref','Customer Ref',"Buyer's reference",'Customer Reference',
                            'order reference','your ref']

                Amount = ['Balance Due','Total Due','Invoice Balance Due','Total Amount Due',
                          'Current invoice balance','Amount Due (USD)','Total in USD',
                          'Total Amount (GBP)','invoice total','TOTAL USD','TOTAL (USD)',
                            'amount due', 'Total EUR','Total (EUR)','Total (GBP)','Invoice subtotal',
                         'USD Total','Total Incl. Vat:','TOTAL €','TOTAL€','PAYMENT DUE',
                         'Total amount','Total Net (EUR)','Total Net(EUR)','Total Net EUR']

                credit_note = ['Credit Note','Credit for Returns','CREDIT-NOTE','ORIGINAL CREDIT MEMO',
                               'CREDIT_NOTE_ALLOWANCE','Credit note','CREDIT NOTE(COPY)','CREDIT MEMO',
                               'Credit Memo','CreditNote',
                                'Return Credit','CREDIT NOTE','RETURN CREDIT MEMO','Credit Reference:',
                               'Cr.Note No','Credit (Copy)','SALES CREDIT NOTE','credit date',
                               'Credit Note No.','Credit Note Number:','Credit Note No.',
                               'RETURN CREDIT','Credit memo','Creditmemo','Cr.Note Date','sales credit']
                inv = []
                inv_date = []
                inv_num = []
                des = []
                vat = []
                unit = []
                del_num = []
                ord_num = []
                amt = []
                final_list = []

                for cr in credit_note:
                    cr_sample = ' '+cr+' '
                    if cr_sample.lower() in final_cred:
                        print(cr)
                        print(os.path.splitext(im)[0]+'.pdf' , 'Credit Note')
                        file_name.append(os.path.splitext(im)[0]+'.pdf')
                        result.append('Credit Note')
                        thresh_list.append('0')
                        break


                else:

                    sp = ' '

                    for a in Invoice:
                        #in_sample = ' '+a+' '
                        if a.lower() in text:
                            print(a)
                            inv.append(a)
                            final_list.append('Invoice')
                            break
                        else:
                            inv.append('-')
                    if len(inv)==inv.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for b in Invoice_Date:
                        #in_date_sample = ' '+b+' '
                        if b.lower() in text:
                            print(b)
                            inv_date.append(b)
                            final_list.append('Invoice_Date')
                            break
                        else:
                            inv_date.append('-')
                    if len(inv_date)==inv_date.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for c in Invoice_Num:
                        #in_num_sample = ' '+c+' '
                        if c.lower() in text:
                            print(c)
                            inv_num.append(c)
                            final_list.append('Invoice_Num')
                            break
                        else:
                            inv_num.append('-')
                    if len(inv_num)==inv_num.count('-'):
                        #print('-------')
                        final_list.append('nil')
                    for d in Description:
                        #in_des_sample = ' '+d+' '
                        if d.lower() in text:
                            print(d)
                            des.append(d)
                            final_list.append('Description')
                            break
                        else:
                            des.append('-')
                    if len(des)==des.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for e in VAT:
                        #in_vat_sample = ' '+e+' '
                        if e.lower() in text:
                            print(e)
                            vat.append(e)
                            final_list.append('VAT')
                            break
                        else:
                            vat.append('-')
                    if len(vat)==vat.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for f in Unit:
                        #in_unit_sample = ' '+f+' '
                        if f.lower() in text:
                            print(f)
                            unit.append(f)
                            final_list.append('Unit')
                            break
                        else:
                            unit.append('-')
                    if len(unit)==unit.count('-'):
                        final_list.append('nil')
                        #print('-------')

                    for g in Deliver_Num:
                        #in_del_sample = ' '+g+' '
                        if g.lower() in text:
                            print(g)
                            del_num.append(g)
                            final_list.append('Deliver_Num')
                            break
                        else:
                            del_num.append('-')
                    if len(del_num)==del_num.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for o in Order_Num:
                        #in_ord_sample = ' '+o+' '
                        if o.lower() in text:
                            print(o)
                            ord_num.append(o)
                            final_list.append('Order_Num')
                            break
                        else:
                            ord_num.append('-')
                    if len(ord_num)==ord_num.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for p in amt:
                        #in_amt_sample = ' '+p+' '
                        if p.lower() in text:
                            print(p)
                            amt.append(p)
                            final_list.append('Amount')
                            break
                        else:
                            amt.append('-')
                    if len(amt)==amt.count('-'):
                        final_list.append('nil')
                        #print('-------')

                    thresh = [15,15,25,10,10,5,5,5,10]

                    threshold = 0
                    for i,j,k in zip(keywords,final_list,thresh):
                        if i == j:
                            threshold = threshold + k

                        #else:
                            #print('--no keyword in image--')

                    #thresh_list.append(threshold)
                    print(threshold)
                    #print(final_list)

                    if val[0][0] >= 0.9:
                        threshold = threshold + 20
                        thresh_list.append(threshold)
                        if threshold >= 65:
                            print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('Invoice')
                        else:
                            print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('UNCLASSIFIED')

                    elif val[0][0] >=0.7 and val[0][0] < 0.9:
                        threshold = threshold + 10
                        thresh_list.append(threshold)
                        if threshold >= 60:
                            print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('Invoice')
                        else:
                            print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('UNCLASSIFIED')

                    else:
                        threshold = threshold + 0
                        thresh_list.append(threshold)
                        if threshold >= 60:
                            print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('Invoice')
                        else:
                            print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('UNCLASSIFIED')
df = pd.DataFrame({'File_name':file_name,'Classified_Result':result,
                   'Threshold':thresh_list})
pd.set_option('display.max_rows',500)
display(df)

df.to_excel('/home/vimalkumar/Documents/softco/test/pdf/testing-6/cnn_results.xlsx')
                
end = time.time()


print(f"Runtime of the program is {end - start}")
        

Document :  1
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613510.pdf
No of Pages :  2
page No: 1
[[9.9998188e-01 4.5955203e-05]]
INVOICE
Description
VAT
Unit
Delivery No
Order No
50
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613510_1.pdf INVOICE
page No: 2
[[9.9999905e-01 8.5374658e-08]]
INVOICE
Description
VAT
Unit
Delivery No
Order No
50
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613510_2.pdf INVOICE
Document :  2
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612788.pdf
No of Pages :  1
page No: 1
[[9.9984246e-01 5.7077408e-04]]
Details
TAX
your ref
25
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612788_1.pdf UNCLASSIFIED
Document :  3
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613177.pdf
No of Pages :  1
page No: 1
[[9.9997789e-01 4.4800386e-06]]
INVOICE
Invoice Date
Invoice No
Description
VAT
Unit
Despatch No
85
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613177_1.pdf INVOICE
Document :  4
CG_28956477-7249-4413-aabd-e045ecd8a19f_3614120.pdf
No of Pages :  2
page No: 1
[[9.9999368e-01 2.0791569e-07]]
IN

[[9.9993247e-01 1.8682127e-05]]
INVOICE
Invoice Date
Invoice No
Description
VAT
Unit
Despatch No
85
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613078_1.pdf INVOICE
Document :  25
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613908.pdf
No of Pages :  2
page No: 1
[[9.999982e-01 4.136601e-07]]
INVOICE
Invoice No
Description
VAT
Unit
Order No
70
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613908_1.pdf INVOICE
page No: 2
[[0.41051453 0.9887018 ]]
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613908_2.pdf UNCLASSIFIED
Document :  26
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612908.pdf
No of Pages :  1
page No: 1
[[9.9999607e-01 5.0039228e-07]]
INVOICE
Description
VAT
Unit
Order No
45
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612908_1.pdf INVOICE
Document :  27
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613647.pdf
No of Pages :  2
page No: 1
[[9.9999940e-01 2.1095437e-07]]
INVOICE
Invoice Date
Invoice No
Description
VAT
Unit
Order No
85
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613647_1.pdf INVOICE
page No: 2


page No: 1
[[0.98372215 0.00413564]]
INVOICE
Product
VAT
Unit
40
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612905_1.pdf UNCLASSIFIED
page No: 2
[[0.01077822 0.9934251 ]]
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612905_2.pdf UNCLASSIFIED
Document :  49
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613070.pdf
No of Pages :  1
page No: 1
[[9.999930e-01 7.182822e-07]]
INVOICE
Invoice Date
Invoice No
Description
VAT
Unit
Despatch No
85
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613070_1.pdf INVOICE
Document :  50
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612896.pdf
No of Pages :  1
page No: 1
[[9.9992782e-01 4.5160846e-06]]
Credit Note
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612896_1.pdf Credit Note
Document :  51
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613848.pdf
No of Pages :  2
page No: 1
[[9.9999976e-01 2.0819416e-08]]
INVOICE
Invoice Date
Invoice No
Description
Unit
Delivery Note
your ref
80
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613848_1.pdf INVOICE
page No: 2
[[0.9954605  0.00342661]]
INV

page No: 1
[[0.6082527 0.3816099]]
INVOICE
Description
VAT
Unit
40
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613431_1.pdf UNCLASSIFIED
Document :  73
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613175.pdf
No of Pages :  1
page No: 1
[[9.9997836e-01 1.7551414e-06]]
INVOICE
Invoice Date
Invoice No
Description
VAT
Unit
Despatch No
85
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613175_1.pdf INVOICE
Document :  74
CG_28956477-7249-4413-aabd-e045ecd8a19f_3614080.pdf
No of Pages :  1
page No: 1
[[9.9996996e-01 6.9136863e-06]]
INVOICE
Description
VAT
Unit
Despatch No
Order No
50
CG_28956477-7249-4413-aabd-e045ecd8a19f_3614080_1.pdf INVOICE
Document :  75
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613582.pdf
No of Pages :  1
page No: 1
[[9.9999547e-01 1.6742972e-07]]
INVOICE
Description
VAT
Unit
Despatch No
Order No
50
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613582_1.pdf INVOICE
Document :  76
CG_28956477-7249-4413-aabd-e045ecd8a19f_3613432.pdf
No of Pages :  1
page No: 1
[[9.9999952e-01 2.8941637e-08]

,File_name,Classified_Result,Threshold
0,CG_28956477-7249-4413-aabd-e045ecd8a19f_361351...,Invoice,70
1,CG_28956477-7249-4413-aabd-e045ecd8a19f_361351...,Invoice,70
2,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,UNCLASSIFIED,45
3,CG_28956477-7249-4413-aabd-e045ecd8a19f_361317...,Invoice,105
4,CG_28956477-7249-4413-aabd-e045ecd8a19f_361412...,Invoice,80
5,CG_28956477-7249-4413-aabd-e045ecd8a19f_361412...,UNCLASSIFIED,60
6,CG_28956477-7249-4413-aabd-e045ecd8a19f_361364...,Invoice,105
7,CG_28956477-7249-4413-aabd-e045ecd8a19f_361364...,UNCLASSIFIED,45
8,CG_28956477-7249-4413-aabd-e045ecd8a19f_361371...,Invoice,70
9,CG_28956477-7249-4413-aabd-e045ecd8a19f_361403...,Invoice,70


Runtime of the program is 605.1083917617798


# Keyword updated version

# All pages title and keyword matching

In [14]:
import cv2
import os
import time
import fitz
import keras
import re
import pytesseract
import pandas as pd
import numpy as np
from pytesseract import Output
from PIL import Image
from termcolor import colored

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image

start = time.time()

in_path = '/home/vimalkumar/Documents/softco/test/pdf/testing-7/pdf'
out_path = '/home/vimalkumar/Documents/softco/test/pdf/testing-7/images'

file_name = []
result = []
thresh_list=[]

count_val = 1
for file in os.listdir(in_path):
    print(colored('Document : ','red', attrs=['bold']),count_val)
    #print('Document : ', count_val)
    count_val += 1
    #print(file)
#     os.makedirs(os.path.join(out_path,os.path.splitext(pdf)[0]))
    #os.makedirs(os.path.join(out_path,os.path.splitext(file)[0]))

#     for file in os.listdir(os.path.join(out_path,os.path.splitext(pdf)[0])):
#         #print(file)
    pdf_file_name = os.path.splitext(file)[0]
    if file.endswith('.tif') or file.endswith('.TIF') or file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.gif') or file.endswith('.jpg') or file.endswith('.JPG')or file.endswith('.webp'):
        #print(file)
        #print(file_name)
        img = Image.open(os.path.join(in_path,file))
        img.seek(0)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(os.path.join(out_path,os.path.splitext(file)[0],pdf_file_name +'.jpeg'))

    if file.endswith('.pdf') or file.endswith('.PDF'):
        print(file)
        
        os.makedirs(os.path.join(out_path,os.path.splitext(file)[0]))

        doc = fitz.open(os.path.join(in_path,file))
        print(colored('No of Pages : ','green', attrs=['bold']),len(doc))
        #print('No of Pages : ',len(doc))
        mat = fitz.Matrix(150 / 72, 150 / 72)
        for k in range(len(doc)):
            page = doc.load_page(k)  # number of page
            pix = page.get_pixmap(matrix=mat)
            #print(type(pix))
            #plt.imshow(pix)

            pix.pil_save(os.path.join(out_path,os.path.splitext(file)[0],pdf_file_name+'_'+str(k+1)+'.jpeg'))

        lst = sorted(os.listdir(os.path.join(out_path,os.path.splitext(file)[0])))
        lst.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

        #for image in lst:
            #print(image)

        model = keras.models.load_model('/home/vimalkumar/Documents/Python_new_1/SOFTCO/CNN Model/invoice_inception_model.h5')

        dic = {0 : 'Invoice',1 : 'Unclassified'}
        #print(dic)

        filepath = os.path.join(out_path,os.path.splitext(file)[0])
        outpath = '/home/vimalkumar/Documents/softco/test/pdf/testing-7/text'


    #     file_name = []
    #     result = []
    #     thresh_list=[]

        count = 1

        for im in lst:
            print(colored('page No:','blue', attrs=['bold']),count)
            #print('page No:',count)
            count = count + 1
            img = image.load_img(os.path.join(filepath,im),
                             target_size=(224,224))
            #plt.imshow(img)
            #plt.show()

            img_arr = image.img_to_array(img)
            img_arr = img_arr/255
            img_arr = np.expand_dims(img_arr,axis=0)
            images = np.vstack([img_arr])
            val = model.predict(images)
            print(val)
            #print(val[0][1])

            a=np.argmax(val,axis=1)
            #print(dic[a[0]])

            if val[0][1] >= 0.95:
                print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                #file_name.append(os.path.splitext(img)[0])
                file_name.append(os.path.splitext(im)[0]+'.pdf')
                result.append('UNCLASSIFIED')
                thresh_list.append('0')

            else:
                f_name = os.path.splitext(im)[0]
                imm = cv2.imread(os.path.join(filepath,im),0)
                
                image_data = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                                       config=r'--oem 3 --psm 6')
                #print(image_data)
                actual_image = cv2.imread(os.path.join(filepath,im))
                gray = cv2.cvtColor(actual_image, cv2.COLOR_BGR2GRAY)
                ret2,thresh= cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                kernel = np.ones((5,5),np.uint8)
                kernel2 = np.ones((3,3),np.uint8)
                marker = cv2.dilate(thresh,kernel,iterations = 1)
                mask=cv2.erode(thresh,kernel,iterations = 1)
                to_test=[]

                while True:
                    tmp=marker.copy()
                    marker=cv2.erode(marker, kernel2)
                    marker=cv2.max(mask, marker)
                    difference = cv2.subtract(tmp, marker)
                    if cv2.countNonZero(difference) == 0:
                        break

                marker_color = cv2.cvtColor(marker, cv2.COLOR_GRAY2BGR)
                cv_image=cv2.bitwise_or(actual_image, marker_color)
                
                bold_text = pytesseract.image_to_string(cv_image, config="--psm 11")



                text=[]
                line_num=[]
                output=[]
                for i in image_data["text"]:
                    text.append(i)

                for j in image_data["line_num"]:
                    line_num.append(j)

                new_list=zip(line_num,text)
                for i in new_list:
                #     print(i)
                    if i[0]>= 0:
                        output.append(i[1])

                # print(output)
                new=" ".join(output)

                image_data_1 = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                               config=r'--oem 3 --psm 11')
                #print(image_data)


                text_1=[]
                line_num_1=[]
                output_1=[]
                for i in image_data_1["text"]:
                    text_1.append(i)

                for j in image_data_1["line_num"]:
                    line_num_1.append(j)

                new_list_1=zip(line_num_1,text_1)
                for i in new_list_1:
                #     print(i)
                    if i[0]>= 0:
                        output_1.append(i[1])

                # print(output)
                new_1=" ".join(output_1)
                #print(new_1)

                final_new = new + new_1
                #print(final_new)

                text_file = open(os.path.join(outpath,f_name + '.txt'), 'w')
                text_file.write(final_new)
                text_file.close()

                #print(new)
                text = final_new.lower()
                #print(text)

                data_df = pd.DataFrame(image_data)

                cred_list = []
                for m in range(15):
                    df_text = data_df[data_df['line_num']==m]

                    for n in df_text['text']:
                        cred_list.append(n)
                cred_list = ' '.join(cred_list).split()
                #print(cred_list)
                cred_out = ''
                for r in cred_list:
                    cred_out= cred_out + ' ' + r

                cred_out = cred_out.lower()
                #print(cred_out)

#                 data_df_1 = pd.DataFrame(image_data_1)

#                 cred_list_1 = []
#                 for m in range(3):
#                     df_text_1 = data_df_1[data_df_1['line_num']==m]

#                     for n in df_text_1['text']:
#                         cred_list_1.append(n)
#                 cred_list_1 = ' '.join(cred_list_1).split()
#                 #print(cred_list_1)
#                 cred_out_1 = ''
#                 for r in cred_list_1:
#                     cred_out_1= cred_out_1 + ' ' + r

#                 cred_out_1 = cred_out_1.lower()
#                 #print(cred_out_1)

                final_cred = cred_out# + cred_out_1
                #print(final_cred)

                keywords = ['Invoice','Invoice_Date','Invoice_Num','Description','VAT','Unit',
                    'Deliver_Num','Order_Num','Amount']

                Invoice = ['INVOICE']

                Invoice_Date = ['Invoice Date','Inv Date','Invoice / Tax Date','DATE/TAX POINT',
                               'Date of invoice',]

                Invoice_Num = ['Invoice Number','Invoice Num','Invoice No','Inv No','Invoice #',
                'Invoice#','Invoice ID','Invoice n°','Invoice N','InvoiceN',
                'Invoice ref','InvoiceRef','Invoice Nr','invoice-no','Sales - Invoice',
                'Sales-Invoice','InvoiceID','Invoice Customer','invoice-n',
                'invoice :','invoice:','number/date','Invoice Number:','invioce no',
                'invioce number','invioce num','invioce #','Invoice/Cr No',
                'Invoice/CrNo','invoice SI','invoiceSI','invoice  SI',
                'INVOICE Doc. No','INVOICE Doc.No','INVOICE Doc No','INVOICE DocNo',
                'invoice(copy)','invoice (copy)']

                Description = ['Description','Product','Details','Materials','Summary',
                               'ACTIVITY','Narrative','Material']

                VAT = ['VAT','TAX','V A T','V.A.T']

                Unit = ['Unit','Quantity','QTY']

                Deliver_Num = ['Deliver Number','Delivery Note','Delivery No','DelNum',
                               'Despatch No',
                              'Delivery Note Number','Del Note','Delivery Order','Delivery Note No']

                Order_Num = ['Order Number','Order No','Order Num','your order','your reference',
                            'Order Ref','Customer Ref',"Buyer's reference",'Customer Reference',
                            'order reference','your ref']

                Amount = ['Balance Due','Total Due','Invoice Balance Due','Total Amount Due',
                          'Current invoice balance','Amount Due (USD)','Total in USD',
                          'Total Amount (GBP)','invoice total','TOTAL USD','TOTAL (USD)',
                            'amount due', 'Total EUR','Total (EUR)','Total (GBP)','Invoice subtotal',
                         'USD Total','Total Incl. Vat:','TOTAL €','TOTAL€','PAYMENT DUE',
                         'Total amount','Total Net (EUR)','Total Net(EUR)','Total Net EUR']

                
                Invoice_title = ['invoice']
                
                
                credit_note = ['Credit Note','Credit for Returns','CREDIT-NOTE','ORIGINAL CREDIT MEMO',
                               'CREDIT_NOTE_ALLOWANCE','Credit note','CREDIT NOTE(COPY)','CREDIT MEMO',
                               'Credit Memo','CreditNote','credit for returns',
                                'Return Credit','CREDIT NOTE','RETURN CREDIT MEMO','Credit Reference:',
                               'Cr.Note No','Credit (Copy)','SALES CREDIT NOTE','credit date',
                               'Credit Note No.','Credit Note Number:','Credit Note No.',
                               'RETURN CREDIT','Credit memo','Creditmemo','Cr.Note Date','sales credit']
                inv = []
                inv_date = []
                inv_num = []
                des = []
                vat = []
                unit = []
                del_num = []
                ord_num = []
                amt = []
                final_list = []

                for cr in credit_note:
                    cr_sample = ' '+cr+' '
                    if cr_sample.lower() in final_cred:
                        print(cr)
                        print(os.path.splitext(im)[0]+'.pdf' , 'Credit Note')
                        file_name.append(os.path.splitext(im)[0]+'.pdf')
                        result.append('Credit Note')
                        thresh_list.append('0')
                        break


                else:

                    sp = ' '

                    for a in Invoice:
                        #in_sample = ' '+a+' '
                        if a.lower() in text:
                            print(a)
                            inv.append(a)
                            final_list.append('Invoice')
                            break
                        else:
                            inv.append('-')
                    if len(inv)==inv.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for b in Invoice_Date:
                        #in_date_sample = ' '+b+' '
                        if b.lower() in text:
                            print(b)
                            inv_date.append(b)
                            final_list.append('Invoice_Date')
                            break
                        else:
                            inv_date.append('-')
                    if len(inv_date)==inv_date.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for c in Invoice_Num:
                        #in_num_sample = ' '+c+' '
                        if c.lower() in text:
                            print(c)
                            inv_num.append(c)
                            final_list.append('Invoice_Num')
                            break
                        else:
                            inv_num.append('-')
                    if len(inv_num)==inv_num.count('-'):
                        #print('-------')
                        final_list.append('nil')
                    for d in Description:
                        #in_des_sample = ' '+d+' '
                        if d.lower() in text:
                            print(d)
                            des.append(d)
                            final_list.append('Description')
                            break
                        else:
                            des.append('-')
                    if len(des)==des.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for e in VAT:
                        #in_vat_sample = ' '+e+' '
                        if e.lower() in text:
                            print(e)
                            vat.append(e)
                            final_list.append('VAT')
                            break
                        else:
                            vat.append('-')
                    if len(vat)==vat.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for f in Unit:
                        #in_unit_sample = ' '+f+' '
                        if f.lower() in text:
                            print(f)
                            unit.append(f)
                            final_list.append('Unit')
                            break
                        else:
                            unit.append('-')
                    if len(unit)==unit.count('-'):
                        final_list.append('nil')
                        #print('-------')

                    for g in Deliver_Num:
                        #in_del_sample = ' '+g+' '
                        if g.lower() in text:
                            print(g)
                            del_num.append(g)
                            final_list.append('Deliver_Num')
                            break
                        else:
                            del_num.append('-')
                    if len(del_num)==del_num.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for o in Order_Num:
                        #in_ord_sample = ' '+o+' '
                        if o.lower() in text:
                            print(o)
                            ord_num.append(o)
                            final_list.append('Order_Num')
                            break
                        else:
                            ord_num.append('-')
                    if len(ord_num)==ord_num.count('-'):
                        final_list.append('nil')
                        #print('-------')
                    for p in Amount:
                        #in_amt_sample = ' '+p+' '
                        if p.lower() in text:
                            print(p)
                            amt.append(p)
                            final_list.append('Amount')
                            break
                        else:
                            amt.append('-')
                    if len(amt)==amt.count('-'):
                        final_list.append('nil')
                        #print('-------')
                        
                    for t in Invoice_title:
                        if re.search(str(t),bold_text.lower()):
                            print('Invoice_title')
                            final_list.append('Invoice_title')
                        else:
                            final_list.append('nil')

                    thresh = [10,15,25,10,10,5,5,5,10,15]

                    threshold = 0
                    for i,j,k in zip(keywords,final_list,thresh):
                        if i == j:
                            threshold = threshold + k

                        #else:
                            #print('--no keyword in image--')

                    #thresh_list.append(threshold)
                    print(threshold)
                    #print(final_list)

                    if val[0][0] >= 0.9:
                        threshold = threshold + 20
                        thresh_list.append(threshold)
                        if threshold >= 65:
                            print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('Invoice')
                        else:
                            print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('UNCLASSIFIED')

                    elif val[0][0] >=0.7 and val[0][0] < 0.9:
                        threshold = threshold + 10
                        thresh_list.append(threshold)
                        if threshold >= 60:
                            print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('Invoice')
                        else:
                            print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('UNCLASSIFIED')

                    else:
                        threshold = threshold + 0
                        thresh_list.append(threshold)
                        if threshold >= 60:
                            print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('Invoice')
                        else:
                            print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                            #file_name.append(os.path.splitext(img)[0])
                            file_name.append(os.path.splitext(im)[0]+'.pdf')
                            result.append('UNCLASSIFIED')
                            
df = pd.DataFrame({'File_name':file_name,'Classified_Result':result,
                   'Threshold':thresh_list})
pd.set_option('display.max_rows',500)
display(df)

df.to_excel('/home/vimalkumar/Documents/softco/test/pdf/testing-7/cnn_results.xlsx')
                
end = time.time()


print(f"Runtime of the program is {end - start}")
        

Document :  1
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612788.pdf
No of Pages :  1
page No: 1
[[9.9984246e-01 5.7077408e-04]]
Details
TAX
your ref
Total (EUR)
35
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612788_1.pdf UNCLASSIFIED
Document :  2
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612899.pdf
No of Pages :  2
page No: 1
[[9.9996126e-01 1.7461118e-05]]
INVOICE
INVOICE Doc. No
Description
VAT
QTY
your reference
Total Net (EUR)
Invoice_title
80
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612899_1.pdf INVOICE
page No: 2
[[9.9980897e-01 6.7556211e-05]]
INVOICE
INVOICE Doc. No
Description
VAT
invoice total
Invoice_title
70
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612899_2.pdf INVOICE
Document :  3
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612800.pdf
No of Pages :  1
page No: 1
[[9.9999607e-01 1.8574088e-06]]
INVOICE
Invoice Date
Invoice No
Details
VAT
Unit
Order No
invoice total
95
CG_28956477-7249-4413-aabd-e045ecd8a19f_3612800_1.pdf INVOICE
Document :  4
CG_28956477-7249-4413-aabd-e045ecd8a19

,File_name,Classified_Result,Threshold
0,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,UNCLASSIFIED,55
1,CG_28956477-7249-4413-aabd-e045ecd8a19f_361289...,Invoice,100
2,CG_28956477-7249-4413-aabd-e045ecd8a19f_361289...,Invoice,90
3,CG_28956477-7249-4413-aabd-e045ecd8a19f_361280...,Invoice,115
4,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,Invoice,70
5,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,UNCLASSIFIED,0
6,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,Invoice,70
7,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,UNCLASSIFIED,0
8,CG_28956477-7249-4413-aabd-e045ecd8a19f_361278...,Invoice,105
9,CG_28956477-7249-4413-aabd-e045ecd8a19f_361290...,Invoice,70


Runtime of the program is 97.04303789138794


# 1st page title match and keyword match

In [6]:
import cv2
import os
import time
import fitz
import keras
import re
import pytesseract
import pandas as pd
import numpy as np
from pytesseract import Output
from PIL import Image
from termcolor import colored

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image

start = time.time()

in_path = '/home/vimalkumar/Documents/softco/National Hardware/Images/Part 1/unclassified'

os.makedirs(os.path.join(in_path,'images'))
os.makedirs(os.path.join(in_path,'text'))


images_path = os.path.join(in_path,'images')
text_file_path = os.path.join(in_path,'text')


file_name = []
result = []
thresh_list=[]

#count_val = 1
for file in os.listdir(in_path):
    #print(colored('Document : ','red', attrs=['bold']),count_val)
    #print('Document : ', count_val)
    #count_val += 1
    #print(file)
#     os.makedirs(os.path.join(out_path,os.path.splitext(pdf)[0]))
    #os.makedirs(os.path.join(out_path,os.path.splitext(file)[0]))

#     for file in os.listdir(os.path.join(out_path,os.path.splitext(pdf)[0])):
#         #print(file)
    pdf_file_name = str(os.path.splitext(file)[0])
    if file.endswith('.tif') or file.endswith('.TIF') or file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.gif') or file.endswith('.jpg') or file.endswith('.JPG')or file.endswith('.webp'):
        #print(file)
        #print(file_name)
        img = Image.open(os.path.join(in_path,file))
        img.seek(0)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(os.path.join(images_path,pdf_file_name +'.jpeg'))

    if file.endswith('.pdf') or file.endswith('.PDF'):
        #print(file)
        
        #os.makedirs(os.path.join(out_path,os.path.splitext(file)[0]))

        doc = fitz.open(os.path.join(in_path,file))
        #print(colored('No of Pages : ','green', attrs=['bold']),len(doc))
        #print('No of Pages : ',len(doc))
        mat = fitz.Matrix(150 / 72, 150 / 72)
        #for k in range(len(doc)):
        page = doc.load_page(0)  # number of page
        pix = page.get_pixmap(matrix=mat)
        #print(type(pix))
        #plt.imshow(pix)

        pix.pil_save(os.path.join(images_path,pdf_file_name+'.jpeg'))

        #lst = sorted(os.listdir(os.path.join(images_path)))
        #lst.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

#         #for image in lst:
#             #print(image)

print('CONVERTED TO IMAGES')

model = keras.models.load_model('/home/vimalkumar/Documents/Python_new_1/SOFTCO/CNN Model/invoice_inception_model.h5')

dic = {0 : 'Invoice',1 : 'Unclassified'}
    #print(dic)

    #filepath = os.path.join(images_path,pdf_file_name)
    #text_file_path = '/home/vimalkumar/Documents/softco/test/pdf/testing-7/text'


#     file_name = []
#     result = []
#     thresh_list=[]

count = 1

for im in os.listdir(images_path):
    print(colored('Document number:','blue', attrs=['bold']),count)
    #print('Document number:',count)
    count = count + 1
    img = image.load_img(os.path.join(images_path,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    img_arr = image.img_to_array(img)
    img_arr = img_arr/255
    img_arr = np.expand_dims(img_arr,axis=0)
    images = np.vstack([img_arr])
    val = model.predict(images)
    print(val)
    #print(val[0][1])

    a=np.argmax(val,axis=1)
    #print(dic[a[0]])

    if val[0][1] >= 0.95:
        print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
        #file_name.append(os.path.splitext(img)[0])
        file_name.append(os.path.splitext(im)[0]+'.pdf')
        result.append('UNCLASSIFIED')
        thresh_list.append('0')

    else:
        f_name = os.path.splitext(im)[0]
        imm = cv2.imread(os.path.join(images_path,im),0)

        image_data = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                               config=r'--oem 3 --psm 6')
        #print(image_data)
        actual_image = cv2.imread(os.path.join(images_path,im))
        gray = cv2.cvtColor(actual_image, cv2.COLOR_BGR2GRAY)
        ret2,thresh= cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((5,5),np.uint8)
        kernel2 = np.ones((3,3),np.uint8)
        marker = cv2.dilate(thresh,kernel,iterations = 1)
        mask=cv2.erode(thresh,kernel,iterations = 1)
        to_test=[]

        while True:
            tmp=marker.copy()
            marker=cv2.erode(marker, kernel2)
            marker=cv2.max(mask, marker)
            difference = cv2.subtract(tmp, marker)
            if cv2.countNonZero(difference) == 0:
                break

        marker_color = cv2.cvtColor(marker, cv2.COLOR_GRAY2BGR)
        cv_image=cv2.bitwise_or(actual_image, marker_color)

        bold_text = pytesseract.image_to_string(cv_image, config="--psm 11")



        text=[]
        line_num=[]
        output=[]
        for i in image_data["text"]:
            text.append(i)

        for j in image_data["line_num"]:
            line_num.append(j)

        new_list=zip(line_num,text)
        for i in new_list:
        #     print(i)
            if i[0]>= 0:
                output.append(i[1])

        # print(output)
        new=" ".join(output)

        image_data_1 = pytesseract.image_to_data(imm, output_type = Output.DICT,
                                       config=r'--oem 3 --psm 11')
        #print(image_data)


        text_1=[]
        line_num_1=[]
        output_1=[]
        for i in image_data_1["text"]:
            text_1.append(i)

        for j in image_data_1["line_num"]:
            line_num_1.append(j)

        new_list_1=zip(line_num_1,text_1)
        for i in new_list_1:
        #     print(i)
            if i[0]>= 0:
                output_1.append(i[1])

        # print(output)
        new_1=" ".join(output_1)
        #print(new_1)

        final_new = new + new_1
        #print(final_new)

        text_file = open(os.path.join(text_file_path,f_name + '.txt'), 'w')
        text_file.write(final_new)
        text_file.close()

        #print(new)
        text = final_new.lower()
        #print(text)

        data_df = pd.DataFrame(image_data)

        cred_list = []
        for m in range(15):
            df_text = data_df[data_df['line_num']==m]

            for n in df_text['text']:
                cred_list.append(n)
        cred_list = ' '.join(cred_list).split()
        #print(cred_list)
        cred_out = ''
        for r in cred_list:
            cred_out= cred_out + ' ' + r

        cred_out = cred_out.lower()
        #print(cred_out)

#                 data_df_1 = pd.DataFrame(image_data_1)

#                 cred_list_1 = []
#                 for m in range(3):
#                     df_text_1 = data_df_1[data_df_1['line_num']==m]

#                     for n in df_text_1['text']:
#                         cred_list_1.append(n)
#                 cred_list_1 = ' '.join(cred_list_1).split()
#                 #print(cred_list_1)
#                 cred_out_1 = ''
#                 for r in cred_list_1:
#                     cred_out_1= cred_out_1 + ' ' + r

#                 cred_out_1 = cred_out_1.lower()
#                 #print(cred_out_1)

        final_cred = cred_out# + cred_out_1
        #print(final_cred)

        keywords = ['Invoice','Invoice_Date','Invoice_Num','Description','VAT','Unit',
            'Deliver_Num','Order_Num','Amount','Invoice_title']

        Invoice = ['INVOICE']

        Invoice_Date = ['Invoice Date','Inv Date','Invoice / Tax Date','DATE/TAX POINT',
                       'Date of invoice',]

        Invoice_Num = ['Invoice Number','Invoice Num','Invoice No','Inv No','Invoice #',
        'Invoice#','Invoice ID','Invoice n°','Invoice N','InvoiceN',
        'Invoice ref','InvoiceRef','Invoice Nr','invoice-no','Sales - Invoice',
        'Sales-Invoice','InvoiceID','Invoice Customer','invoice-n',
        'invoice :','invoice:','number/date','Invoice Number:','invioce no',
        'invioce number','invioce num','invioce #','Invoice/Cr No',
        'Invoice/CrNo','invoice SI','invoiceSI','invoice  SI',
        'INVOICE Doc. No','INVOICE Doc.No','INVOICE Doc No','INVOICE DocNo',
        'invoice(copy)','invoice (copy)']

        Description = ['Description','Product','Details','Materials','Summary',
                       'ACTIVITY','Narrative','Material']

        VAT = ['VAT','TAX','V A T','V.A.T']

        Unit = ['Unit','Quantity','QTY']

        Deliver_Num = ['Deliver Number','Delivery Note','Delivery No','DelNum',
                       'Despatch No',
                      'Delivery Note Number','Del Note','Delivery Order','Delivery Note No']

        Order_Num = ['Order Number','Order No','Order Num','your order','your reference',
                    'Order Ref','Customer Ref',"Buyer's reference",'Customer Reference',
                    'order reference','your ref']

        Amount = ['Balance Due','Total Due','Invoice Balance Due','Total Amount Due',
                  'Current invoice balance','Amount Due (USD)','Total in USD',
                  'Total Amount (GBP)','invoice total','TOTAL USD','TOTAL (USD)',
                    'amount due', 'Total EUR','Total (EUR)','Total (GBP)','Invoice subtotal',
                 'USD Total','Total Incl. Vat:','TOTAL €','TOTAL€','PAYMENT DUE',
                 'Total amount','Total Net (EUR)','Total Net(EUR)','Total Net EUR',
                 'Total-EUR','Total - EUR']


        Invoice_title = ['invoice','i n v o i c e']


        credit_note = ['Credit Note','Credit for Returns','CREDIT-NOTE','ORIGINAL CREDIT MEMO',
                       'CREDIT_NOTE_ALLOWANCE','Credit note','CREDIT NOTE(COPY)','CREDIT MEMO',
                       'Credit Memo','CreditNote',
                        'Return Credit','CREDIT NOTE','RETURN CREDIT MEMO','Credit Reference:',
                       'Cr.Note No','Credit (Copy)','SALES CREDIT NOTE','credit date',
                       'Credit Note No.','Credit Note Number:','Credit Note No.',
                       'RETURN CREDIT','Credit memo','Creditmemo','Cr.Note Date',
                       'sales credit','credit for returns']
        
        
        
        inv = []
        inv_date = []
        inv_num = []
        des = []
        vat = []
        unit = []
        del_num = []
        ord_num = []
        amt = []
        final_list = []

        for cr in credit_note:
            cr_sample = ' '+cr+' '
            if cr_sample.lower() in final_cred:
                print(cr)
                print(os.path.splitext(im)[0]+'.pdf' , 'Credit Note')
                file_name.append(os.path.splitext(im)[0]+'.pdf')
                result.append('Credit Note')
                thresh_list.append('0')
                break


        else:

            sp = ' '

            for a in Invoice:
                #in_sample = ' '+a+' '
                if a.lower() in text:
                    print(a)
                    inv.append(a)
                    final_list.append('Invoice')
                    break
                else:
                    inv.append('-')
            if len(inv)==inv.count('-'):
                final_list.append('nil')
                #print('-------')
            for b in Invoice_Date:
                #in_date_sample = ' '+b+' '
                if b.lower() in text:
                    print(b)
                    inv_date.append(b)
                    final_list.append('Invoice_Date')
                    break
                else:
                    inv_date.append('-')
            if len(inv_date)==inv_date.count('-'):
                final_list.append('nil')
                #print('-------')
            for c in Invoice_Num:
                #in_num_sample = ' '+c+' '
                if c.lower() in text:
                    print(c)
                    inv_num.append(c)
                    final_list.append('Invoice_Num')
                    break
                else:
                    inv_num.append('-')
            if len(inv_num)==inv_num.count('-'):
                #print('-------')
                final_list.append('nil')
            for d in Description:
                #in_des_sample = ' '+d+' '
                if d.lower() in text:
                    print(d)
                    des.append(d)
                    final_list.append('Description')
                    break
                else:
                    des.append('-')
            if len(des)==des.count('-'):
                final_list.append('nil')
                #print('-------')
            for e in VAT:
                #in_vat_sample = ' '+e+' '
                if e.lower() in text:
                    print(e)
                    vat.append(e)
                    final_list.append('VAT')
                    break
                else:
                    vat.append('-')
            if len(vat)==vat.count('-'):
                final_list.append('nil')
                #print('-------')
            for f in Unit:
                #in_unit_sample = ' '+f+' '
                if f.lower() in text:
                    print(f)
                    unit.append(f)
                    final_list.append('Unit')
                    break
                else:
                    unit.append('-')
            if len(unit)==unit.count('-'):
                final_list.append('nil')
                #print('-------')

            for g in Deliver_Num:
                #in_del_sample = ' '+g+' '
                if g.lower() in text:
                    print(g)
                    del_num.append(g)
                    final_list.append('Deliver_Num')
                    break
                else:
                    del_num.append('-')
            if len(del_num)==del_num.count('-'):
                final_list.append('nil')
                #print('-------')
            for o in Order_Num:
                #in_ord_sample = ' '+o+' '
                if o.lower() in text:
                    print(o)
                    ord_num.append(o)
                    final_list.append('Order_Num')
                    break
                else:
                    ord_num.append('-')
            if len(ord_num)==ord_num.count('-'):
                final_list.append('nil')
                #print('-------')
            for p in Amount:
                #in_amt_sample = ' '+p+' '
                if p.lower() in text:
                    print(p)
                    amt.append(p)
                    final_list.append('Amount')
                    break
                else:
                    amt.append('-')
            if len(amt)==amt.count('-'):
                final_list.append('nil')
                #print('-------')

            for t in Invoice_title:
                if re.search(str(t),bold_text.lower()):
                    print('Invoice_title')
                    final_list.append('Invoice_title')
                    break
                else:
                    final_list.append('nil')

            thresh = [10,15,25,10,10,5,5,5,10,15]

            threshold = 0
            for i,j,k in zip(keywords,final_list,thresh):
                if i == j:
                    threshold = threshold + k

                #else:
                    #print('--no keyword in image--')

            #thresh_list.append(threshold)
            print(threshold)
            #print(final_list)

            if val[0][0] >= 0.9:
                threshold = threshold + 20
                thresh_list.append(threshold)
                if threshold >= 65:
                    print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('Invoice')
                else:
                    print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('UNCLASSIFIED')

            elif val[0][0] >=0.7 and val[0][0] < 0.9:
                threshold = threshold + 10
                thresh_list.append(threshold)
                if threshold >= 60:
                    print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('Invoice')
                else:
                    print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('UNCLASSIFIED')

            else:
                threshold = threshold + 0
                thresh_list.append(threshold)
                if threshold >= 60:
                    print(os.path.splitext(im)[0]+'.pdf' ,'INVOICE')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('Invoice')
                else:
                    print(os.path.splitext(im)[0]+'.pdf' ,'UNCLASSIFIED')
                    #file_name.append(os.path.splitext(img)[0])
                    file_name.append(os.path.splitext(im)[0]+'.pdf')
                    result.append('UNCLASSIFIED')

df = pd.DataFrame({'File_name':file_name,'Classified_Result':result,
           'Threshold':thresh_list})
pd.set_option('display.max_rows',500)
display(df)

df.to_excel(os.path.join(in_path,'cnn_results.xlsx'))

end = time.time()


print(f"Runtime of the program is {end - start}")


CONVERTED TO IMAGES
Document number: 1
[[0.04969624 0.9892261 ]]
NH1_169193_357456.pdf UNCLASSIFIED
Document number: 2
[[0.04969624 0.9892261 ]]
NH1_169193_357455.pdf UNCLASSIFIED
Document number: 3
[[0.01164207 0.99961305]]
NH1_169193_357442.pdf UNCLASSIFIED


,File_name,Classified_Result,Threshold
0,NH1_169193_357456.pdf,UNCLASSIFIED,0
1,NH1_169193_357455.pdf,UNCLASSIFIED,0
2,NH1_169193_357442.pdf,UNCLASSIFIED,0


Runtime of the program is 3.715766191482544
